In [1]:
import itertools
from collections import OrderedDict

import jax
import numpy as np
import pandas as pd
from jax import lax
from jax import numpy as jnp
from jax import random
from jax.random import PRNGKey

import kerax
from kerax import utils
from kerax.layers import (Activation, Add, Concatenate, Conv2D, Dense, Flatten,
                          Input, Layer, MultiHeadAttention)
from kerax.models import Model


2022-04-05 17:17:02.335534: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
df = pd.read_csv('train.csv')

In [3]:
labels = df['label'].values

In [4]:
df.drop('label', axis=1, inplace=True)

In [5]:
imgs = df.values.reshape((df.shape[0], 28, 28, 1)).astype('float32')

In [6]:
cat_train_y = utils.to_categorical(labels, 10)

/home/hazem/miniconda3/envs/jax/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py:5154: UserWarning: Explicitly requested dtype int requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  lax_internal._check_user_dtype_supported(dtype, "astype")


In [12]:
kerax.backend.enable_jit_execution(False)

In [13]:
inputs = Input(shape=(28, 28, 1))
conv1 = Conv2D(64, 3, padding='same', key=PRNGKey(100))(inputs)
act1 = Activation('relu')(conv1)
conv3 = Conv2D(128, 3, padding='same', key=PRNGKey(104))(act1)
act2 = Activation('relu')(conv3)
conv4 = Conv2D(128, 3, padding='same', key=PRNGKey(105), activation='relu')(act2)
add = Add()([act2, conv4])
flatten = Flatten()(add)
dense1 = Dense(512, activation='relu')(flatten)
output = Dense(10, activation='softmax')(dense1)

In [14]:
model = Model(inputs, output)

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
model.fit(imgs, cat_train_y, epochs=10, batch_size=128)

 40%|===============                         | 131/328 [02:21<03:32,  1.08s/it, Loss=14.894131]


KeyboardInterrupt: 